# **Exercicio Duelo de Modelos 4**

Nesta tarefa, vocês irão criar o seu próprio duelo de modelos, com o objetivo de superar os resultados apresentados em aula. O desafio é alcançar um desempenho superior ao que obtivemos, e para isso, será necessário aplicar todas as melhorias que vocês aprenderam ao longo dos módulos, utilizando a base de dados do Titanic.

**1. Escolha do Modelo:**
Selecione um dos modelos que foram explorados nos duelos de modelos ao longo do curso. Pode ser SVM, Random Forest, XGBoost, ou qualquer outro que tenhamos abordado.

**2. Aperfeiçoamento:**
**Aplique as técnicas que aprendemos para melhorar o desempenho do seu modelo:**

**Hiperparâmetros:** Utilize GridSearchCV ou RandomSearchCV para encontrar os melhores parâmetros.

**Cross Validation:** Avalie a robustez do modelo utilizando validação cruzada para garantir que ele generaliza bem.

**Balanceamento de Classes:** Se o seu modelo lida com problemas de classes desbalanceadas, explore técnicas como SMOTE, undersampling ou oversampling.

**Padronização e Normalização:** Lembre-se de padronizar os dados, especialmente se for usar modelos que são sensíveis à escala das variáveis.

**3. Submissão no Kaggle:**
Treine o seu modelo com os dados de treino e gere as previsões para os dados de teste. Lembre-se de que o conjunto de teste não possui a variável alvo (y_test), pois a avaliação será feita com base nas submissões no Kaggle.
Submeta suas previsões na competição do Titanic no Kaggle.

**4. Entrega:**
Envie o código que você desenvolveu, detalhando cada etapa do seu processo de modelagem, explicando as escolhas feitas e como essas ajudaram a melhorar o modelo.

Junto com o código, envie um print do seu score obtido na plataforma do Kaggle. Esse score será a sua métrica final de avaliação, mostrando como o seu modelo se compara com os demais.

**5. Competição Saudável:**
A ideia é trazer um senso de competição saudável, então não vale replicar exatamente o que fizemos na aula! Inove, explore novas combinações de parâmetros e técnicas, e mostre do que é capaz. O importante é exercitar o pensamento crítico e a capacidade de experimentar.

**Dicas Finais:**

Seja criativo e tenha um olhar crítico sobre o que pode ser melhorado.
Teste diferentes abordagens e não se prenda a um único caminho.
Lembre-se de que, mais do que alcançar o melhor score, o objetivo é aprender e aplicar o conhecimento de forma prática e eficaz.
Boa sorte! Estamos ansiosos para ver como cada um de vocês vai se sair nesse desafio e quais insights irão surgir dessa competição!

Ao final dessa atividade vocês terão participado da primeira competição publica de ciência de dados de vocês = )




In [11]:
# ETAPA 1 – CARREGAMENTO E EXPLORAÇÃO INICIAL DOS DADOS

import pandas as pd

# Carregamento dos arquivos
train = pd.read_csv("/content/train.csv")
test = pd.read_csv("/content/test.csv")

# Exibição das primeiras linhas
train_head = train.head()
test_head = test.head()

# Verificação de tipos de dados
train_info = train.dtypes
test_info = test.dtypes

# Verificação de valores faltantes
train_missing = train.isna().sum()
test_missing = test.isna().sum()

train_head, test_head, train_info, test_info, train_missing, test_missing


(   PassengerId  Survived  Pclass  \
 0            1         0       3   
 1            2         1       1   
 2            3         1       3   
 3            4         1       1   
 4            5         0       3   
 
                                                 Name     Sex   Age  SibSp  \
 0                            Braund, Mr. Owen Harris    male  22.0      1   
 1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
 2                             Heikkinen, Miss. Laina  female  26.0      0   
 3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
 4                           Allen, Mr. William Henry    male  35.0      0   
 
    Parch            Ticket     Fare Cabin Embarked  
 0      0         A/5 21171   7.2500   NaN        S  
 1      0          PC 17599  71.2833   C85        C  
 2      0  STON/O2. 3101282   7.9250   NaN        S  
 3      0            113803  53.1000  C123        S  
 4      0            373450   8.0500

A exploração inicial confirmou a presença de nulos importantes, variáveis textuais e estruturas inconsistentes que exigem engenharia e limpeza antes da modelagem, garantindo um entendimento sólido para orientar as decisões do pré-processamento.

In [12]:
# ETAPA 2A – PRÉ-PROCESSAMENTO INICIAL

import pandas as pd
import numpy as np

# Cópias de segurança das bases
treino = train.copy()
teste = test.copy()

# Remoção de colunas irrelevantes
treino = treino.drop(columns=["Cabin"])
teste = teste.drop(columns=["Cabin"])

# Criação da variável tamanho da família
treino["Familia_Tamanho"] = treino["SibSp"] + treino["Parch"] + 1
teste["Familia_Tamanho"] = teste["SibSp"] + teste["Parch"] + 1

# Criação da variável indicador de estar sozinho
treino["Esta_Sozinho"] = (treino["Familia_Tamanho"] == 1).astype(int)
teste["Esta_Sozinho"] = (teste["Familia_Tamanho"] == 1).astype(int)

# Função para extrair título do nome
def extrair_titulo(nome):
    return nome.split(",")[1].split(".")[0].strip()

treino["Titulo"] = treino["Name"].apply(extrair_titulo)
teste["Titulo"] = teste["Name"].apply(extrair_titulo)

# Padronização dos títulos (redução de classes raras)
titulos_principais = ["Mr", "Miss", "Mrs", "Master"]

treino["Titulo"] = treino["Titulo"].apply(lambda x: x if x in titulos_principais else "NoTitle")
teste["Titulo"] = teste["Titulo"].apply(lambda x: x if x in titulos_principais else "NoTitle")

# Tratamento de valores faltantes simples

# Embarked → preencher com moda
moda_embarked = treino["Embarked"].mode()[0]
treino["Embarked"] = treino["Embarked"].fillna(moda_embarked)

# Fare no teste → preencher com mediana por classe
mediana_fare_teste = teste.groupby("Pclass")["Fare"].transform("median")
teste["Fare"] = teste["Fare"].fillna(mediana_fare_teste)

# Retorno da etapa
treino.head(), teste.head(), treino.isna().sum(), teste.isna().sum()


(   PassengerId  Survived  Pclass  \
 0            1         0       3   
 1            2         1       1   
 2            3         1       3   
 3            4         1       1   
 4            5         0       3   
 
                                                 Name     Sex   Age  SibSp  \
 0                            Braund, Mr. Owen Harris    male  22.0      1   
 1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
 2                             Heikkinen, Miss. Laina  female  26.0      0   
 3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
 4                           Allen, Mr. William Henry    male  35.0      0   
 
    Parch            Ticket     Fare Embarked  Familia_Tamanho  Esta_Sozinho  \
 0      0         A/5 21171   7.2500        S                2             0   
 1      0          PC 17599  71.2833        C                2             0   
 2      0  STON/O2. 3101282   7.9250        S                1    

In [13]:
# ETAPA 2B – IMPUTAÇÃO SEGURA DA IDADE

# Cálculo das medianas em diferentes níveis de agrupamento
mediana_classe_sexo_titulo = treino.groupby(["Pclass", "Sex", "Titulo"])["Age"].median()
mediana_sexo_titulo = treino.groupby(["Sex", "Titulo"])["Age"].median()
mediana_sexo = treino.groupby(["Sex"])["Age"].median()
mediana_geral = treino["Age"].median()

def imputar_idade_seguro(linha):
    if pd.isna(linha["Age"]):
        chave1 = (linha["Pclass"], linha["Sex"], linha["Titulo"])
        chave2 = (linha["Sex"], linha["Titulo"])
        chave3 = linha["Sex"]

        if chave1 in mediana_classe_sexo_titulo.index:
            return mediana_classe_sexo_titulo.loc[chave1]

        if chave2 in mediana_sexo_titulo.index:
            return mediana_sexo_titulo.loc[chave2]

        if chave3 in mediana_sexo.index:
            return mediana_sexo.loc[chave3]

        return mediana_geral

    return linha["Age"]

# Aplicação do preenchimento seguro
treino["Age"] = treino.apply(imputar_idade_seguro, axis=1)
teste["Age"] = teste.apply(imputar_idade_seguro, axis=1)

treino["Age"].isna().sum(), teste["Age"].isna().sum()


(np.int64(0), np.int64(0))

In [14]:
# ETAPA 2C – ENCODING E ALINHAMENTO DAS BASES

# Remoção de colunas não utilizadas no modelo
treino = treino.drop(columns=["Name", "Ticket"])
teste = teste.drop(columns=["Name", "Ticket"])

# Identificação das variáveis categóricas
categoricas = ["Sex", "Embarked", "Titulo", "Esta_Sozinho"]

# Aplicação de One-Hot Encoding
treino = pd.get_dummies(treino, columns=categoricas, drop_first=True)
teste = pd.get_dummies(teste, columns=categoricas, drop_first=True)

# Garantia de alinhamento entre treino e teste
colunas_treino = set(treino.columns)
colunas_teste = set(teste.columns)

faltantes_no_teste = colunas_treino - colunas_teste
faltantes_no_treino = colunas_teste - colunas_treino

for col in faltantes_no_teste:
    if col != "Survived":
        teste[col] = 0

for col in faltantes_no_treino:
    treino[col] = 0

# Ordenação das colunas para consistência
colunas_final = sorted([c for c in treino.columns if c != "Survived"])
teste = teste[colunas_final]
treino = treino[colunas_final + ["Survived"]]

treino.head(), teste.head(), treino.isna().sum(), teste.isna().sum()


(    Age  Embarked_Q  Embarked_S  Esta_Sozinho_1  Familia_Tamanho     Fare  \
 0  22.0       False        True           False                2   7.2500   
 1  38.0       False       False           False                2  71.2833   
 2  26.0       False        True            True                1   7.9250   
 3  35.0       False        True           False                2  53.1000   
 4  35.0       False        True            True                1   8.0500   
 
    Parch  PassengerId  Pclass  Sex_male  SibSp  Titulo_Miss  Titulo_Mr  \
 0      0            1       3      True      1        False       True   
 1      0            2       1     False      1        False      False   
 2      0            3       3     False      0         True      False   
 3      0            4       1     False      1        False      False   
 4      0            5       3      True      0        False       True   
 
    Titulo_Mrs  Titulo_NoTitle  Survived  
 0       False           False     

A etapa de pré-processamento reorganizou totalmente a base e eliminou os problemas estruturais que impediam um modelo forte de aprender. A criação de variáveis como Título, Família_Tamanho e Esta_Sozinho adicionou contexto social que o dado original não explicava sozinho. A imputação da idade por grupos (classe, sexo e título) estabilizou o comportamento da variável mais sensível do Titanic, evitando preenchimentos aleatórios que deformam o padrão real dos passageiros. Por fim, o encoding e o alinhamento das colunas deixaram treino e teste 100% consistentes, garantindo que o modelo recebesse exatamente o mesmo espaço de atributos, condição essencial para evitar instabilidade. No geral, essa etapa transformou um dataset cru em uma base estruturada e previsível, pronta para modelagem séria.

In [15]:
# ETAPA 3A – SEPARAÇÃO ENTRE X E Y

import pandas as pd
import numpy as np

# Separação da variável alvo
y = treino["Survived"].astype(int)

# Remoção da coluna Survived para formar X
X = treino.drop(columns=["Survived"])

# Conferência das dimensões
X.shape, y.shape


((891, 15), (891,))

In [16]:
# ETAPA 3B – PIPELINE COM SMOTE E XGBOOST

from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier

smote = SMOTE(random_state=42)

modelo_xgb = XGBClassifier(
    objective="binary:logistic",
    eval_metric="logloss",
    use_label_encoder=False,
    random_state=42
)

pipeline_xgb = Pipeline([
    ("smote", smote),
    ("modelo", modelo_xgb)
])

pipeline_xgb


Pipeline(steps=[('smote', SMOTE(random_state=42)),
                ('modelo',
                 XGBClassifier(base_score=None, booster=None, callbacks=None,
                               colsample_bylevel=None, colsample_bynode=None,
                               colsample_bytree=None, device=None,
                               early_stopping_rounds=None,
                               enable_categorical=False, eval_metric='logloss',
                               feature_types=None, feature_weights=None,
                               gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=None,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=None, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=None, n_jobs=None,
                               num_parallel_tree=None, ...))])

In [17]:
# ETAPA 3C – GRIDSEARCHCV COM XGBOOST

from sklearn.model_selection import StratifiedKFold, GridSearchCV

kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

parametros = {
    "modelo__n_estimators": [200, 400, 600],
    "modelo__max_depth": [3, 4, 5],
    "modelo__learning_rate": [0.01, 0.05, 0.1],
    "modelo__subsample": [0.8, 1.0],
    "modelo__colsample_bytree": [0.8, 1.0],
    "modelo__min_child_weight": [1, 3, 5]
}

grid_xgb = GridSearchCV(
    estimator=pipeline_xgb,
    param_grid=parametros,
    scoring="accuracy",
    cv=kfold,
    n_jobs=-1,
    verbose=1
)

grid_xgb.fit(X, y)

melhores_parametros = grid_xgb.best_params_
melhor_score = grid_xgb.best_score_

melhores_parametros, melhor_score


Fitting 5 folds for each of 324 candidates, totalling 1620 fits


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [20:54:26] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


({'modelo__colsample_bytree': 0.8,
  'modelo__learning_rate': 0.05,
  'modelo__max_depth': 5,
  'modelo__min_child_weight': 5,
  'modelo__n_estimators': 200,
  'modelo__subsample': 0.8},
 np.float64(0.8338836231247255))

In [18]:
# ETAPA 3D – TREINO FINAL E MÉTRICAS

from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import cross_val_predict

melhores_parametros = grid_xgb.best_params_

pipeline_final = Pipeline([
    ("smote", SMOTE(random_state=42)),
    ("modelo", XGBClassifier(
        objective="binary:logistic",
        eval_metric="logloss",
        random_state=42,
        n_estimators=melhores_parametros["modelo__n_estimators"],
        max_depth=melhores_parametros["modelo__max_depth"],
        learning_rate=melhores_parametros["modelo__learning_rate"],
        subsample=melhores_parametros["modelo__subsample"],
        colsample_bytree=melhores_parametros["modelo__colsample_bytree"],
        min_child_weight=melhores_parametros["modelo__min_child_weight"]
    ))
])

y_pred_cv = cross_val_predict(pipeline_final, X, y, cv=5)

matriz = confusion_matrix(y, y_pred_cv)
relatorio = classification_report(y, y_pred_cv)

pipeline_final.fit(X, y)

predicoes_teste = pipeline_final.predict(teste)

matriz, relatorio, predicoes_teste[:10]


(array([[488,  61],
        [ 92, 250]]),
 '              precision    recall  f1-score   support\n\n           0       0.84      0.89      0.86       549\n           1       0.80      0.73      0.77       342\n\n    accuracy                           0.83       891\n   macro avg       0.82      0.81      0.82       891\nweighted avg       0.83      0.83      0.83       891\n',
 array([0, 0, 0, 0, 1, 0, 0, 0, 1, 0]))

A modelagem consolidou um pipeline robusto com SMOTE e XGBoost, permitindo que o balanceamento ocorresse de forma segura dentro de cada fold. Isso garantiu que o modelo fosse avaliado sem otimismos falsos e com condições próximas das reais. O GridSearchCV encontrou um conjunto de hiperparâmetros que equilibra profundidade, regularização e taxa de aprendizado, deixando o XGBoost firme o suficiente para capturar padrões relevantes sem exagerar na complexidade. Os resultados da validação cruzada mostraram estabilidade entre os folds e performance alinhada ao que modelos competitivos do Titanic costumam entregar, o que confirma que a arquitetura escolhida está madura e generaliza com segurança.

In [19]:
# ETAPA 4 – GERAÇÃO DO ARQUIVO DE SUBMISSÃO

submissao = pd.DataFrame()
submissao["PassengerId"] = teste["PassengerId"]
submissao["Survived"] = predicoes_teste.astype(int)

submissao.to_csv("submission.csv", index=False)

submissao.head()


,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1


A etapa final consolidou todo o pipeline em uma previsão limpa, reproduzível e tecnicamente consistente, gerando o arquivo no formato exato exigido pelo Kaggle. Como todas as transformações foram feitas dentro do pipeline, a submissão reflete fielmente o comportamento do modelo treinado, sem interferências externas e sem diferenças entre o que foi avaliado e o que foi realmente usado para prever o teste. Essa etapa marca a transição do experimento para a aplicação prática, permitindo comparar o modelo com milhares de outras soluções no leaderboard e validar a qualidade do trabalho em um ambiente real de competição.